In [9]:

# ====
# 1-Minute FX Data Downloader – Dukascopy (Incremental Update)
# ====

from datetime import datetime, timedelta, timezone
from pathlib import Path
import dukascopy_python as dp
from dukascopy_python.instruments import (
    INSTRUMENT_FX_MAJORS_EUR_USD,
    INSTRUMENT_FX_MAJORS_GBP_USD,
    INSTRUMENT_FX_MAJORS_USD_JPY
)
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm.auto import tqdm

# ---- AYARLAR ----
PAIRS = {
    "EURUSD": INSTRUMENT_FX_MAJORS_EUR_USD,
    "GBPUSD": INSTRUMENT_FX_MAJORS_GBP_USD,
    "USDJPY": INSTRUMENT_FX_MAJORS_USD_JPY
}

INTERVAL = dp.INTERVAL_MIN_1   # 1-dakika
SIDE    = dp.OFFER_SIDE_BID    # sadece bid fiyatları

OUTPUT_DIR = Path(r"C:/DukascopyData")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
# ----

def get_last_timestamp(file_path: Path) -> datetime:
    """Mevcut dosyanın son timestamp'ini döndürür."""
    try:
        if file_path.exists():
            df = pd.read_parquet(file_path)
            if not df.empty:
                # Index'in timezone-aware olup olmadığını kontrol et
                last_ts = df.index.max()
                if hasattr(last_ts, 'tz') and last_ts.tz is not None:
                    # Timezone-aware ise timezone-naive'e çevir (UTC olarak)
                    return last_ts.tz_convert('UTC').replace(tzinfo=None)
                else:
                    # Zaten timezone-naive ise direkt döndür
                    return last_ts.to_pydatetime()
    except Exception as e:
        print(f"Dosya okuma hatası {file_path}: {e}")
    
    # Dosya yoksa veya hata varsa başlangıç tarihi döndür
    return datetime(2015, 1, 1)

def fetch_and_update_pair(symbol: str, instrument):
    """Belirtilen parite için incremental update yapar."""
    out_path = OUTPUT_DIR / f"{symbol}_1min.parquet"
    
    # Mevcut verinin son tarihini al
    last_date = get_last_timestamp(out_path)
    
    # Yeni veriler için başlangıç tarihi (son tarihten 1 dakika sonra)
    start_date = last_date + timedelta(minutes=1)
    # Bugünün başlangıcına kadar al (timezone-naive)
    end_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    
    print(f"\n{symbol}:")
    print(f"  Mevcut veri son tarihi: {last_date}")
    print(f"  Yeni veri aralığı: {start_date} → {end_date}")
    
    # Eğer güncel veri varsa indirme yapma
    if start_date >= end_date:
        print(f"  ✓ {symbol} zaten güncel!")
        return
    
    try:
        # Yeni verileri indir
        new_df = dp.fetch(instrument, INTERVAL, SIDE, start_date, end_date)
        
        if new_df.empty:
            print(f"  ⚠ {symbol} için yeni veri bulunamadı")
            return
        
        # Mevcut veriyi yükle (varsa)
        if out_path.exists():
            existing_df = pd.read_parquet(out_path)
            # Yeni veriyi ekle
            combined_df = pd.concat([existing_df, new_df[["close"]].rename(columns={"close": symbol})])
            # Duplicate index'leri kaldır (son değeri tut)
            combined_df = combined_df[~combined_df.index.duplicated(keep='last')]
            combined_df = combined_df.sort_index()
        else:
            # İlk kez indirme
            combined_df = new_df[["close"]].rename(columns={"close": symbol})
        
        # Parquet'e kaydet
        pq.write_table(
            pa.Table.from_pandas(combined_df),
            out_path,
            compression="snappy"
        )
        
        print(f"  ✓ {len(new_df):,} yeni satır eklendi")
        print(f"  ✓ Toplam: {len(combined_df):,} satır → {out_path}")
        
    except Exception as e:
        print(f"  ✗ {symbol} indirme hatası: {e}")

# --- İNCREMENTAL UPDATE döngüsü ----
print("🔄 Incremental data update başlıyor...")

for sym, inst in tqdm(PAIRS.items(), desc="Updating pairs"):
    fetch_and_update_pair(sym, inst)

print("\n✔︎ Tüm güncellemeler tamamlandı!")

# Sonuçları kontrol et
print("\n📊 Güncel veri durumu:")
for symbol in PAIRS.keys():
    file_path = OUTPUT_DIR / f"{symbol}_1min.parquet"
    if file_path.exists():
        df = pd.read_parquet(file_path)
        print(f"  {symbol}: {len(df):,} satır | Son: {df.index.max()}")

🔄 Incremental data update başlıyor...


Updating pairs:   0%|          | 0/3 [00:00<?, ?it/s]


EURUSD:
  Mevcut veri son tarihi: 2025-06-17 21:00:00
  Yeni veri aralığı: 2025-06-17 21:01:00 → 2025-06-18 00:00:00
  ⚠ EURUSD için yeni veri bulunamadı

GBPUSD:
  Mevcut veri son tarihi: 2025-06-17 20:59:00
  Yeni veri aralığı: 2025-06-17 21:00:00 → 2025-06-18 00:00:00
  ⚠ GBPUSD için yeni veri bulunamadı

USDJPY:
  Mevcut veri son tarihi: 2015-01-01 00:00:00
  Yeni veri aralığı: 2015-01-01 00:01:00 → 2025-06-18 00:00:00


INFO:DUKASCRIPT:current timestamp :2015-01-30T20:00:00
INFO:DUKASCRIPT:current timestamp :2015-03-02T16:16:00
INFO:DUKASCRIPT:current timestamp :2015-03-31T11:39:00
INFO:DUKASCRIPT:current timestamp :2015-04-29T08:27:00
INFO:DUKASCRIPT:current timestamp :2015-05-28T06:21:00
INFO:DUKASCRIPT:current timestamp :2015-06-26T02:56:00
INFO:DUKASCRIPT:current timestamp :2015-07-27T00:22:00
INFO:DUKASCRIPT:current timestamp :2015-08-24T22:20:00
INFO:DUKASCRIPT:current timestamp :2015-09-22T19:25:00
INFO:DUKASCRIPT:current timestamp :2015-10-21T16:23:00
INFO:DUKASCRIPT:current timestamp :2015-11-19T13:53:00
INFO:DUKASCRIPT:current timestamp :2015-12-18T10:34:00
INFO:DUKASCRIPT:current timestamp :2016-01-19T22:55:00
INFO:DUKASCRIPT:current timestamp :2016-02-17T19:06:00
INFO:DUKASCRIPT:current timestamp :2016-03-17T14:23:00
INFO:DUKASCRIPT:current timestamp :2016-04-15T14:35:00
INFO:DUKASCRIPT:current timestamp :2016-05-16T11:15:00
INFO:DUKASCRIPT:current timestamp :2016-06-14T08:33:00
INFO:DUKAS

  ✓ 3,897,088 yeni satır eklendi
  ✓ Toplam: 3,897,088 satır → C:\DukascopyData\USDJPY_1min.parquet

✔︎ Tüm güncellemeler tamamlandı!

📊 Güncel veri durumu:
  EURUSD: 3,898,418 satır | Son: 2025-06-17 21:00:00+00:00
  GBPUSD: 3,900,028 satır | Son: 2025-06-17 20:59:00+00:00
  USDJPY: 3,897,088 satır | Son: 2025-06-17 20:59:00+00:00


In [12]:
import pandas as pd

# EURUSD verilerini oku
eur = pd.read_parquet(r"C:\DukascopyData\EURUSD_1min.parquet")

# GBPUSD verilerini oku
gbp = pd.read_parquet(r"C:\DukascopyData\GBPUSD_1min.parquet")

# USDJPY verilerini oku
usd_jpy = pd.read_parquet(r"C:\DukascopyData\USDJPY_1min.parquet")

print("EURUSD")
print(eur.head())
print(eur.tail())
print(f"\nToplam satır sayısı: {len(eur):,}")
print(f"Veri aralığı: {eur.index.min()} → {eur.index.max()}")

print("GBPUSD")
print(gbp.head())
print(gbp.tail())
print(f"\nToplam satır sayısı: {len(gbp):,}")
print(f"Veri aralığı: {gbp.index.min()} → {gbp.index.max()}")

print("USDJPY")
print(usd_jpy.head())
print(usd_jpy.tail())
print(f"\nToplam satır sayısı: {len(usd_jpy):,}")
print(f"Veri aralığı: {usd_jpy.index.min()} → {usd_jpy.index.max()}")


EURUSD
                            EURUSD
timestamp                         
2014-12-31 21:00:00+00:00  1.21005
2014-12-31 21:01:00+00:00  1.21005
2014-12-31 21:02:00+00:00  1.21005
2014-12-31 21:03:00+00:00  1.20996
2014-12-31 21:04:00+00:00  1.20996
                            EURUSD
timestamp                         
2025-06-17 20:56:00+00:00  1.14836
2025-06-17 20:57:00+00:00  1.14815
2025-06-17 20:58:00+00:00  1.14796
2025-06-17 20:59:00+00:00  1.14794
2025-06-17 21:00:00+00:00  1.14773

Toplam satır sayısı: 3,898,418
Veri aralığı: 2014-12-31 21:00:00+00:00 → 2025-06-17 21:00:00+00:00
GBPUSD
                            GBPUSD
timestamp                         
2014-12-31 21:00:00+00:00  1.55833
2014-12-31 21:01:00+00:00  1.55848
2014-12-31 21:02:00+00:00  1.55871
2014-12-31 21:03:00+00:00  1.55873
2014-12-31 21:04:00+00:00  1.55861
                            GBPUSD
timestamp                         
2025-06-17 20:55:00+00:00  1.34251
2025-06-17 20:56:00+00:00  1.34253
2025-06-17 